In [ ]:
#import libraries
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
import math
import requests
from bs4 import BeautifulSoup
from datetime import datetime

#### Only use Selenium for extracting href

In [142]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
 
# def scrape_url(base_url):
 
#     # Load WebDriver and navigate to the page url.
#     # This will open a browser window.
#     driver = webdriver.Chrome("C:/Users/Gabriela/Programming/courses/Python_mega_course_app4_to_10/app7_Real_Estate_Webscraper/chromedriver.exe")
#     driver.get(base_url)
     
#     #initiate list to store property-specififc URLs in    
#     urls = []
 
#     #  get number of results --> used to determine over how many pages to iterate
#     property_nr = int(driver.find_element_by_class_name('results-label').get_attribute('data-count'))
#     driver.quit()
#     print(f"Number of properties: {property_nr}")
    
#     # calculate number of pages to scroll over and round up float 
#     total_pages = math.ceil(property_nr/100) #Note use math.ceil() instead of np.ceil() to return int instead of float
    
#     #iterate over total_pages and add info to base_URL
#     for i in range(1,total_pages+1):
#         url = base_url + f"&p={i}"
#         driver = webdriver.Chrome("C:/Users/Gabriela/Programming/courses/Python_mega_course_app4_to_10/app7_Real_Estate_Webscraper/chromedriver.exe")
#         driver.get(url)
        
#         #initiate scroll_count in order to avoid infinite loop on final page
#         scroll_count =0
        
#         # First scroll to the end of the table by sending Page Down keypresses to
#         # the browser window.
#         print(f'Scrolling through page {i} of {total_pages}')
#         while driver.find_elements_by_tag_name('h3')[-1].text != "You've Seen the next 100 properties." \
#         and driver.find_elements_by_tag_name('h3')[-1].text != "You've Seen the first 100 properties." \
#         and scroll_count < 200:
#             # Find the first element on the page, so we can scroll down using the
#             # element object's send_keys() method
#             scroll_count +=1
#             elem = driver.find_element_by_class_name('listing-price')
#             elem.send_keys(Keys.PAGE_DOWN)

#         # Once the whole table has loaded, grab all the visible links. 
#         print(f'Saving property URLs for page {i} to list.')
#         visible_links = driver.find_elements_by_class_name('listing-price')
#         for link in visible_links:
#             urls.append(link.get_attribute('href'))
#         if i == total_pages:
#             pass
#         else:
#             print(f"{len(urls)} properties stored to list.")

#         driver.quit()
    
#     print(f"Done. {len(urls)} property URLs saved to list.")
    
#     return urls

#### Use Selenium to extract more information

In [248]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
 
def scrape_url(base_url):
     
    #start timer
    start = datetime.now()
        
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Chrome("C:/Users/Gabriela/Programming/courses/Python_mega_course_app4_to_10/app7_Real_Estate_Webscraper/chromedriver.exe")
    driver.get(base_url)
     
    #initiate list to store property-specififc URLs in    
    urls = []
 
    #  get number of results --> used to determine over how many pages to iterate
    property_nr = int(driver.find_element_by_class_name('results-label').get_attribute('data-count'))
    driver.quit()
    print(f"Number of properties: {property_nr}")
    
    # calculate number of pages to scroll over and round up float 
    total_pages = math.ceil(property_nr/100) #Note use math.ceil() instead of np.ceil() to return int instead of float
    
    #iterate over total_pages and add info to base_URL
    for i in range(1,total_pages+1):
        url = base_url + f"&p={i}"
        driver = webdriver.Chrome("C:/Users/Gabriela/Programming/courses/Python_mega_course_app4_to_10/app7_Real_Estate_Webscraper/chromedriver.exe")
        driver.get(url)
        
        #initiate scroll_count in order to avoid infinite loop on final page
        scroll_count =0
        
        # First scroll to the end of the table by sending Page Down keypresses to
        # the browser window.
        print(f'Scrolling through page {i} of {total_pages}')
        while driver.find_elements_by_tag_name('h3')[-1].text != "You've Seen the next 100 properties." \
        and driver.find_elements_by_tag_name('h3')[-1].text != "You've Seen the first 100 properties." \
        and scroll_count < 200:
            # Find the first element on the page, so we can scroll down using the
            # element object's send_keys() method
            scroll_count +=1
            elem = driver.find_element_by_class_name('listing-price')
            elem.send_keys(Keys.PAGE_DOWN)

        # Once the whole table has loaded, grab all the visible links. 
        print(f'Saving property URLs for page {i} to list.')
        visible_links = driver.find_elements_by_class_name('property-card-primary-info')
        
        for link in visible_links:
            info = {}
            #get mandatory information
            href = link.find_element_by_class_name("listing-price").get_attribute('href')
            
            price = link.find_element_by_class_name("listing-price").text.strip()
            try:
                price = int(''.join(c for c in price if c.isdigit())) #string non-numeric symbold and convert price to int
            except:
                price = price
                
            try:
                street = link.find_element_by_class_name("property-address").get_attribute('title')
            except:
                street = "-"
            try:
                location = link.find_element_by_class_name("property-city").text.strip()
                neighborhood = location.split()[0]
                zip_ = location.split()[-1]
            except:
                location = "-"
                neighborhood = "-"
                zip_ = "-"
              
            #get optional information
            try:
                beds = int(link.find_element_by_class_name("property-beds").find_element_by_tag_name("strong").text)
            except:
                beds = "-"
            try:
                baths = int(link.find_element_by_class_name("property-baths").find_element_by_tag_name("strong").text)
            except:
                baths = "-"
            try:
                size = int(link.find_element_by_class_name("property-sqft").find_element_by_tag_name("strong").text)
            except:
                size = "-"

                
            #add information to info-dict
            info['href'] = href
            info['Price'] = price
            info['Bedrooms'] = beds
            info['Bathrooms'] = baths
            info['Size'] = size
            info['Street'] = street
            info['zip'] = zip_
            info['address'] = str(street) + ', ' + str(location)
            
            #append to urls-list
            urls.append(info)
        if i == total_pages:
            pass
        else:
            print(f"{len(urls)} properties stored to list.")

        driver.quit()
    
    print(f"Done. {len(urls)} property URLs saved to list.")
    
    #end timer
    end = datetime.now()
    #calculate elapsed time
    print(f"\nScraping took {end-start}.")
    
    return urls

In [ ]:
urls = scrape_url("https://www.century21.com/real-estate/brooklyn-ny/LCNYBROOKLYN/?sa=CNYASTORIA%2CCNYBREEZYPOINT%2CCNYELMHURST%2CCNYHOWARDBEACH%2CCNYJACKSONHEIGHTS%2CCNYLONGISLANDCITY%2CCNYMASPETH%2CCNYMIDDLEVILLAGE%2CCNYOZONEPARK%2CCNYREGOPARK%2CCNYRIDGEWOOD%2CCNYSUNNYSIDE%2CCNYWOODHAVEN%2CCNYWOODSIDE&pr=%7C350000")

Number of properties: 807
Scrolling through page 1 of 9
Saving property URLs for page 1 to list.
100 properties stored to list.
Scrolling through page 2 of 9
Saving property URLs for page 2 to list.
200 properties stored to list.
Scrolling through page 3 of 9
Saving property URLs for page 3 to list.
300 properties stored to list.
Scrolling through page 4 of 9
Saving property URLs for page 4 to list.
400 properties stored to list.
Scrolling through page 5 of 9
Saving property URLs for page 5 to list.
500 properties stored to list.
Scrolling through page 6 of 9
Saving property URLs for page 6 to list.


In [231]:
print(len(urls))
urls[0:3]

807


[{'href': 'https://www.century21.com/property/7401-4th-avenue-e7-brooklyn-ny-11209-CBR51643402',
  'Price': 320000,
  'Bedrooms': 1,
  'Bathrooms': 1,
  'Size': nan,
  'Street': '7401 4th Avenue #E7',
  'zip': '11209',
  'address': '7401 4th Avenue #E7, Brooklyn NY 11209'},
 {'href': 'https://www.century21.com/property/1170-ocean-parkway-8-g-brooklyn-ny-11230-REN014069992',
  'Price': 205000,
  'Bedrooms': 1,
  'Bathrooms': 1,
  'Size': nan,
  'Street': '1170 Ocean Parkway 8-G',
  'zip': '11230',
  'address': '1170 Ocean Parkway 8-G, Brooklyn NY 11230'},
 {'href': 'https://www.century21.com/property/86-29-shore-parkway-unit-0008-howard-beach-ny-11414-C2182174151',
  'Price': 315000,
  'Bedrooms': 2,
  'Bathrooms': 1,
  'Size': nan,
  'Street': '86-29 Shore Parkway Unit 0008',
  'zip': '11414',
  'address': '86-29 Shore Parkway Unit 0008, Howard Beach NY 11414'}]

#### Use requests and BeautifulSoup to get more specific information from property-hrefs

In [247]:
def info_to_table(info_dict):
    url = info_dict['href']
    # get property-website content
    r = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    
    #check if listing is for sale
    if soup.find("div", {"class":"pdp-listing-type"}).text.lower() == 'for sale' \
        or soup.find("div", {"class":"pdp-listing-type"}).text.lower() == 'sale':

        #get additional information
        features = soup.find("ul", {"class":"property-features"}).find_all("li")
        #iterate over features and extract info
        feats = ""
        for feature in features:
            feats += feature.text + "\n" 
        info_dict['Property Features'] = feats

        return(info_dict)
    

{'href': 'https://www.century21.com/property/86-29-shore-parkway-unit-0008-howard-beach-ny-11414-C2182174151', 'Price': 315000, 'Bedrooms': 2, 'Bathrooms': 1, 'Size': '-', 'Street': '86-29 Shore Parkway Unit 0008', 'zip': '11414', 'address': '86-29 Shore Parkway Unit 0008, Howard Beach NY 11414', 'Property Features': 'Appliances: Dryer, Range / Oven, Refrigerator, Washer\nFlooring: Hardwood\nSewer: City\nStyle: Other Style\nWater: City Water\n'}


####  Save information in df

In [ ]:
df = pd.DataFrame()

for prop in info_dict[:6]:
    new = info_to_table(info_dict)
    try:
        df.append(new, ingnore_index=True)
    except:
        pass

In [211]:
driver = webdriver.Chrome("C:/Users/Gabriela/Programming/courses/Python_mega_course_app4_to_10/app7_Real_Estate_Webscraper/chromedriver.exe")
driver.get('https://www.century21.com/property/7401-4th-avenue-e7-brooklyn-ny-11209-CBR51643402')
print(driver.find_element_by_class_name('pdp-info-price').find_element_by_tag_name('h5').ext)
driver.quit()

AttributeError: 'WebElement' object has no attribute 'ext'